<a href="https://colab.research.google.com/github/last-brain-cell/cryptographic-attacks/blob/main/Chosen_Ciphertext_Attack_(on_RSA_style_Public_Key).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This demonstrates how a chosen ciphertext can reveal information about the plaintext using RSA properties.

In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.1 MB/s eta 0:00:00


In [3]:
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes
from math import gcd

# Retry until e is coprime with φ(n)
def generate_rsa_keys(bits=8, e=3):
    while True:
        p = getPrime(bits)
        q = getPrime(bits)
        if p == q:
            continue
        phi = (p - 1) * (q - 1)
        if gcd(e, phi) == 1:
            n = p * q
            d = inverse(e, phi)
            return (e, n), (d, n), (p, q)

# Generate RSA keys
public_key, private_key, primes = generate_rsa_keys()
e, n = public_key
d, _ = private_key
p, q = primes

print(f"Public key (e={e}, n={n})")
print(f"Private key (d={d})")
print(f"Primes used: p={p}, q={q}")


Public key (e=3, n=22879)
Private key (d=15051)
Primes used: p=167, q=137


In [4]:
def encrypt_rsa(m, e, n):
    return pow(m, e, n)

def decrypt_rsa(c, d, n):
    return pow(c, d, n)

# Example message
message = b"OK"
m = bytes_to_long(message)
c = encrypt_rsa(m, e, n)
decrypted_m = decrypt_rsa(c, d, n)

print("Original Message:", message)
print("Encrypted Ciphertext:", c)
print("Decrypted Message:", long_to_bytes(decrypted_m))

Original Message: b'OK'
Encrypted Ciphertext: 14496
Decrypted Message: b'OK'


In [5]:
# Attacker intercepts ciphertext c
# Picks multiplier s
s = 2
c_prime = (pow(s, e, n) * c) % n  # Send c' to decrypt

# Victim (oracle) decrypts
m_prime = decrypt_rsa(c_prime, d, n)

# Attacker recovers original m
s_inv = inverse(s, n)
recovered_m = (m_prime * s_inv) % n

print("Recovered Message:", long_to_bytes(recovered_m))

Recovered Message: b'OK'
